# Capstone Project: Week 1

Tutoring Service Location Search

1. Introduction/Business Problem:
Our goal is to ﬁnd the best location (areas/neighborhood) to start a tutoring service business in Ontario (Brampton), Canada. We need to ﬁnd areas with the schools with the largest enrollment as well as the lowest number of existing tutoring services (oﬀered nearby). The presence of schools determines existence of clients (students), the lack of existing tutoring services means a lack of competition for our prospective business owner. We also consider schools with a lower percentage of low-income families or a higher percentage of parents with university education which indicates that parents can aﬀord the tutorial services offered by the business owner. The process is to specialize, ﬁltering for Elementary or Secondary School, cities, and such.

2. Data:
Source data is publicly available from the government of Ontario at the following link: https://www.ontario.ca/data/schoolinformation-and-student-demographics
This data-set provides info about almost 5,000 schools all over Ontario, with the location coordinates and the number of students enrolled. Using the location data along with foursquare, a venue search engine, we may search for tutoring services near each of the schools. These will be categorized by the number of services nearby. 


In [1]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium as fol
import pandas as pd
pd.options.display.max_columns= None
pd.set_option('display.max_colwidth', -1)
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import requests

def foursq_search(lats, lngs, query, limit=50, radius=3000 ):
    '''Search Foursquare at the interable coordinates [lats/lngs] given for the given [query]. 
    Return a list of jsons containing the results'''
    res=[]
    CLIENT_ID = '025VQIOHO0CSWOHDXX2GMBUPVHA4ILTSWWVFNZQL5V1EQRIT' # your Foursquare ID
    CLIENT_SECRET = 'MVYDCCA1PZRQH204PYM5X3EKBBOYMPHTSPVF1C1PN0DLZTHJ' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version

    base_url= 'https://api.foursquare.com/v2/venues/search?'
    
    for lat, lng in zip( lats, lngs):
        url= base_url + '&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                VERSION,
                lat,
                lng,
                radius,
                query,
                limit)
        try:
            result= requests.get(url).json()
        except:
            print('Error searching: {},{}. Assigning 0 venues.'.format(lat, lng))
            result= { 'response':{} }
        res.append(result)
    return res

def extract_results( results, amt=[], venues=[], unique_venues= [], specs= [] , excls= []):
    '''Extract/update from each json in the list of [results] the [amt] of venues and the [unique_venues]. 
    Optionally return only venues containing a string in the [specs] list and not one in the [exclude] list.
    Return a list with the number of venues in each result and a dataframe with the lat/lng/name of each unique venue'''

    for i, result in enumerate( results ):
        
        if len( amt ) < len( results ): 
            amt.append( 0 )
        
        # non-empty result
        if result['response'] != {}:
                
            # iterate through the venues in the response
            for venue in result['response']['venues']:

                # make a tuple of the lat/lng/name of each venue
                temp_venue= ( venue['location']['lat'], venue['location']['lng'], venue['name'] )

                # if the specifics list is nonempty check that at least one of the strings is in the venue name
                # if there are specifics and they aren't in the venue name move on to the next venue
                # similar process if a string in the inclusion list is present
                if ( specs != [] ) & ( np.array([spec.lower() in temp_venue[2].lower() for spec in specs] ).sum() == 0): 
                    continue
                if ( excls != [] ) & ( np.array([excl.lower() in temp_venue[2].lower() for excl in excls] ).sum() != 0): 
                    continue

                amt[i]+= 1 
                
                # if we haven't encountered this venue, add it to our unique venue list
                if not temp_venue in unique_venues: 
                    unique_venues+= [ temp_venue ]

                    
    return amt, unique_venues



def plot_points( lats, lngs , radii=[], colors=[], labels=[], opacities=[], toner=False,zoom=12, prev_map= None):
    '''Plot/add coordinates [lats/lngs] with optional [radii],[colors],[labels]. Optinally update a [prev_map].
    Return a map.'''
    
    pt_amt= len( lats )
    
    # check initial conditions    
    make_popups= lambda labels: [ fol.Popup( l, parse_html=True) for l in labels ] \
                                if len( labels ) == pt_amt \
                                else [None] * pt_amt
    check_radii= lambda radii: radii if len( radii ) == pt_amt else [1] * pt_amt
    check_colors= lambda colors: colors if len( colors ) == pt_amt else ['black'] * pt_amt
    check_opacities= lambda opacities: opacities if len( opacities ) == pt_amt else [1] * pt_amt
    
    popups= make_popups( labels )
    radii= check_radii( radii )
    colors= check_colors( colors )
    opacities= check_opacities( opacities )
    
    # if there was no previous map make a new one
    if prev_map == None:
        center= [ lats.mean(), lngs.mean() ]
        tiles= 'Stamen Toner' if toner else 'OpenStreetMap'
        prev_map= fol.Map( location=center, zoom_start=zoom, control_scale=True, tiles=tiles)
        
    for lat, lng, r, color, op, popup in zip(lats, lngs, radii, colors, opacities, popups):
        fol.Circle(
            location=[lat,lng],
            radius=r,
            color=color,
            popup= popup,
            fill=True,
            fill_color=color,
            fill_opacity=op
        ).add_to(prev_map)

    return prev_map

### Import raw data & Data Preparation
Source data is publicly available from the government of Ontario at the following link: https://www.ontario.ca/data/schoolinformation-and-student-demographics
This data-set provides info about almost 5,000 schools all over Ontario, with the location coordinates and the number of students enrolled. Using the location data along with foursquare, a venue search engine, we may search for tutoring services near each of the schools. These will be categorized by the number of services nearby. 


In [2]:
# ontario has free data
data= pd.read_excel('https://files.ontario.ca/opendata/sif_data_table_2015_2016_en.xlsx')

# drop what seem to be keys as well as irrelevant/redundant columns
data.drop(['Board Number','Board Type','School Number', 'Province', 'Municipality','School Website','Board Website','Building Suite','P.O. Box'],axis=1,inplace=True)

# title case the city column for ease
data['City'] = data['City'].apply(lambda x: x.title())

# take only english speaking elementary and secondary schools into account
data= data[ data['School Language'] == 'English' ]
data.drop('School Language', axis=1, inplace=True)

data= data[ (data['School Level'] == 'Elementary') | (data['School Level'] == 'Secondary') ]
data.drop('School Level', axis=1, inplace=True)

print('The data has {} rows & {} cols.'.format(data.shape[0],data.shape[1]))

data.head()

The data has 4449 rows & 42 cols.


,Board Name,School Name,School Type,School Special Condition Code,Grade Range,Street,City,Postal Code,Phone Number,Fax Number,Enrolment,Latitude,Longitude,Percentage of Students Whose First Language Is Not English,Percentage of Students Whose First Language Is Not French,Percentage of Students Who Are New to Canada from a Non-English Speaking Country,Percentage of Students Who Are New to Canada from a Non-French Speaking Country,Percentage of Students Receiving Special Education Services,Percentage of Students Identified as Gifted,Percentage of Grade 3 Students Achieving the Provincial Standard in Reading,Change in Grade 3 Reading Achievement Over Three Years,Percentage of Grade 3 Students Achieving the Provincial Standard in Writing,Change in Grade 3 Writing Acheivement Over Three Years,Percentage of Grade 3 Students Achieving the Provincial Standard in Mathematics,Change in Grade 3 Mathematics Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Reading,Change in Grade 6 Reading Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Writing,Change in Grade 6 Writing Acheivement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Mathematics,Change in Grade 6 Mathematics Achievement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Academic Mathematics,Change in Grade 9 Academic Mathematics Acheivement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Applied Mathematics,Change in Grade 9 Applied Mathematics Achievement Over Three Years,Percentage of Students That Passed the Grade 10 OSSLT on Their First Attempt,Change in Grade 10 OSSLT Literacy Achievement Over Three Years,Percentage of Children Who Live in Low-Income Households,Percentage of Students Whose Parents Have Some Unviersity Education,Percentage of JK-Grade 3 Classes With 20 Students or Fewer,Percentage of JK-Grade 3 Classes With 23 Students or Fewer,Extract Date
0,Algoma DSB,Algoma Education Connection Secondary School,Public,Alternative,9-12,550 NORTHERN AVENUE,Sault Ste. Marie,P6B4J4,NaN,NaN,236.0,46.53477,-84.30772,NaN,100,NaN,NaN,18.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/D,NaN,N/R,NaN,N/R,NaN,33.88,SP,NaN,NaN,Dec-04-17
1,Algoma DSB,Anna McCrea Public School,Public,Not applicable,JK-8,250 Mark,Sault Ste Marie,P6A3M7,705-945-7106,705-945-7221,168.0,46.50593,-84.28732,SP,100,SP,SP,15.5,NaN,0.77,NaN,0.58,NaN,0.81,NaN,0.8,NaN,0.67,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1,20.97,1.0,1.0,Dec-04-17
2,Algoma DSB,Arthur Henderson Public School,Public,Not applicable,JK-8,2 Henderson,Bruce Mines,P0R1C0,705-785-3483,705-785-3220,101.0,46.30183,-83.78020,SP,100,NaN,NaN,11.9,NaN,0.38,NaN,0.31,NaN,0.46,NaN,N/D,NaN,N/D,NaN,N/D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.42,SP,0.0,1.0,Dec-04-17
3,Algoma DSB,Ben R McMullin Public School,Public,Not applicable,JK-8,24 Paradise,Sault Ste Marie,P6B5K2,705-945-7108,705-945-7205,189.0,46.52455,-84.29804,SP,100,SP,SP,13.8,SP,0.44,NaN,0.38,NaN,0.44,NaN,0.74,NaN,0.65,NaN,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.9,14.95,1.0,1.0,Dec-04-17
4,Algoma DSB,Blind River Public School,Public,Not applicable,JK-8,19 Hanes,Blind River,P0R1B0,705-356-7752,705-356-0271,187.0,46.18454,-82.95760,SP,100,SP,SP,23,NaN,0.5,NaN,0.36,NaN,0.5,NaN,0.52,NaN,0.65,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.36,10.7,1.0,1.0,Dec-04-17


### Clean up our data into the dataframe we will use
Create a dataframe including the most useful columns from the original data, we also get rid of null values and replace them with the average for the column.

In [3]:
# extract only the columns we want
cols= ['School Name','Enrolment','Latitude','Longitude','City']
school_df= data[cols].copy()

#these are the numerical portions of the original data
pct_df= data.iloc[:,-5:-3]

school_df= pd.concat( [school_df, pct_df], axis=1 , sort=True )

# change the column names to make them easier to work with
school_df.columns= ['school','enrol','lat','lng','city','pct_low_income', 'pct_uni_parents']

# drop all entries with null in any of the specified columns
school_df.dropna(subset= ['school','enrol','lat','lng','city'], inplace=True)

# make null entries the average for the numerical data
for col in school_df:
    if not col in ['school', 'enrol', 'lat', 'lng' ,'city']:
        avg= 0
        num_entries= 0
        for val in school_df[col].values:
            if (not val in ['SP','N/R','N/D']) & (val == val):
                avg+= val
                num_entries+= 1
        avg= avg / num_entries
        school_df[col].replace( ['SP','N/R','N/D', np.nan], avg, inplace=True )

print('The schools dataframe has {} rows & {} cols.'.format(school_df.shape[0],school_df.shape[1]))
school_df.head()

The schools dataframe has 4357 rows & 7 cols.


,school,enrol,lat,lng,city,pct_low_income,pct_uni_parents
0,Algoma Education Connection Secondary School,236.0,46.53477,-84.30772,Sault Ste. Marie,33.88,24.372899
1,Anna McCrea Public School,168.0,46.50593,-84.28732,Sault Ste Marie,8.10,20.970000
2,Arthur Henderson Public School,101.0,46.30183,-83.78020,Bruce Mines,13.42,24.372899
3,Ben R McMullin Public School,189.0,46.52455,-84.29804,Sault Ste Marie,27.90,14.950000
4,Blind River Public School,187.0,46.18454,-82.95760,Blind River,22.36,10.700000


### Narrow our scope to Brampton

In [4]:
brampton_df= school_df[ school_df.city == 'Brampton' ].copy()

# drop columns we dont need
brampton_df.drop('city', axis=1,inplace=True)
brampton_df.reset_index(drop=True, inplace=True)

print('There are {} schools in Brampton'.format(brampton_df.shape[0]))
brampton_df.head()

There are 169 schools in Brampton


,school,enrol,lat,lng,pct_low_income,pct_uni_parents
0,Bishop Francis Allen Catholic School,358.0,43.66624,-79.74619,23.25,29.09
1,Cardinal Ambrozic Catholic Secondary School,1332.0,43.78772,-79.68312,16.58,31.78
2,Cardinal Leger Secondary School,1118.0,43.68409,-79.75050,20.87,11.33
3,Cardinal Newman Catholic School,504.0,43.72155,-79.69919,22.76,15.02
4,Father C W Sullivan Catholic School,301.0,43.70595,-79.74661,17.87,11.46
